In [1]:
import io
import requests
import pandas as pd

/Users/edupmon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Análise da exportação brasileira de vinhos de mesa (2009 - 2023)

In [2]:
# Origem dos dados de exportação: http://vitibrasil.cnpuv.embrapa.br

'''Os dados podem ser acessados, diretamente através da página, passando como
   parâmetro o ano e as opções dos dados que se deseja acessar.
   Neste caso: &opcao=opt_06&subopcao=subopt_01 - Tabela de exportação de
                                                  vinhos de mesa
   A tabela de índice 3 contém os dados de interesse e as duas últimas linhas
   podem ser desconsideradas (uma linha de rodapé e uma linha de total)'''

# Exportação dos dados de origem para CSV
# (para preservar os dados do estudo caso a página seja alterada ou removida)

url = 'http://vitibrasil.cnpuv.embrapa.br/index.php?'

# Safras de interesse
ano_inicial = 2009
ano_final = 2023

for ano in range(ano_inicial, ano_final + 1):
    # Parâmetros da URL
    url += f'ano={ano}&opcao=opt_06&subopcao=subopt_01'
    # Leitura dos dados
    df = pd.read_html(io.StringIO(requests.get(url).text))[3][:-2]
    # Renomeando as colunas
    df.rename(columns={'Países': 'pais', 'Quantidade (Kg)': 'qtd_litros',
                       'Valor (US$)': 'vlr_usd'}, inplace=True)
    # Substituindo os caracteres '-' e '.' nas colunas de qtd e vlr
    df['qtd_litros'] = df['qtd_litros'].str.replace('-|\.', '', regex = True)
    df['vlr_usd'] = df['vlr_usd'].str.replace('-|\.', '', regex = True)
    # Coluna ano
    df['ano'] = ano
    # Exportando o CSV
    df.to_csv(f'dados/exp-vinho-{ano}.csv', index=False)